## **SKYBUG INTERNSHIP**

## **AI INTERN**

## **TASK - 4**

## **Done By : A S V Dhanush**


## Create a simple recommendation system that suggests items to users based on their preferences.

## You can use techniques like collaborative filtering or content-based filtering to recommend movies,books, or products to users.

### **Data Preparation**


In [42]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load the anime dataset
anime = pd.read_csv('./dataset_folder/anime.csv')
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [3]:
# Load the rating dataset
ratings = pd.read_csv('./dataset_folder/rating.csv')
ratings.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


### **Data Preprocessing**

In [4]:
# For computing reasons I'm limiting the dataframe length to 10,000 users
ratings = ratings[ratings.user_id <= 10000]

In [5]:
# Preprocess the rating dataset (handle -1 ratings as NaN)
ratings['rating'] = ratings['rating'].replace(-1, 0)

In [6]:
# Delete duplicated items
ratings = ratings.drop_duplicates(['user_id', 'anime_id'])

In [7]:
# Create a user-item interaction matrix
user_item_matrix = ratings.pivot(index='user_id', columns='anime_id', values='rating')

### **User-Based Collaborative Filtering**
Impementing user-based collaborative filtering by calculating the similarity between users based on their ratings.

In [8]:
# Calculate user similarity using cosine similarity
user_similarity = cosine_similarity(user_item_matrix.fillna(0))

#### **Making Recommendations**
To make recommendations for a target user, we identify users similar to the target user and recommend anime that they have rated positively.

In [9]:
def user_based_recommendations(user_id, user_item_matrix, user_similarity, n=5):
    # Get the similarity scores for the target user
    user_scores = user_similarity[user_id - 1]
    
    # Sort users by similarity in descending order
    similar_users = sorted(enumerate(user_scores), key=lambda x: x[1], reverse=True)
    
    # Initialize a list to store recommended anime
    recommended_anime = []
    
    for user, score in similar_users[1:]:  # Exclude the target user
        # Get the anime the similar user has rated positively
        rated_anime = user_item_matrix.loc[user_item_matrix.index[user - 1]]
        positively_rated_anime = rated_anime[rated_anime >= 6].index
        
        # Exclude anime the target user has already rated
        target_user_rated_anime = user_item_matrix.loc[user_id].dropna().index
        recommended_anime.extend(set(positively_rated_anime) - set(target_user_rated_anime))
        
        # Limit the number of recommendations
        if len(recommended_anime) >= n:
            break
    
    return recommended_anime[:n]


### **Item-Based Collaborative Filtering**
**Item Similarity**
Implementing item-based collaborative filtering by calculating the similarity between anime based on user ratings.

In [10]:
# Calculate item similarity using cosine similarity
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)

#### **Making Recommendations**
To make item-based recommendations, we identify anime similar to those the user has already rated positively.

In [11]:
def item_based_recommendations(user_id, user_item_matrix, item_similarity, n=5):
    # Get the anime the user has rated positively
    positively_rated_anime = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] >= 6].index
    
    # Initialize a list to store recommended anime
    recommended_anime = []
    
    for anime_id in positively_rated_anime:
        # Get anime similar to the liked anime
        similar_anime = item_similarity[anime_id - 1]
        
        # Sort similar anime by similarity in descending order
        similar_anime_sorted = sorted(enumerate(similar_anime), key=lambda x: x[1], reverse=True)
        
        for anime, score in similar_anime_sorted[1:]:  # Exclude the liked anime
            # Exclude anime the user has already rated
            user_rated_anime = user_item_matrix.loc[user_id].dropna().index
            if anime not in user_rated_anime:
                recommended_anime.append(anime)
            
            # Limit the number of recommendations
            if len(recommended_anime) >= n:
                break
    
    return recommended_anime[:n]

### **Test Recommendation**

In [38]:

user_test_ids =[9,22,69]
recommendations = []

#Testing using the helper functions
for i,id in enumerate(user_test_ids):
    sublist = []
    user_recommendations = user_based_recommendations(id, user_item_matrix, user_similarity, n=5)
    item_recommendations = item_based_recommendations(id, user_item_matrix, item_similarity, n=5)
    sublist.append(user_recommendations)
    sublist.append(item_recommendations)
    recommendations.append(sublist)
    


In [41]:
# Display the recommendations

for i in recommendations:
    print(f"User-Based Recommendations for User {user_test_ids[recommendations.index(i)]}:")
    for anime_id in user_recommendations:
        anime_name = anime.loc[anime['anime_id'] == anime_id, 'name'].values[0]
        print(f"- {anime_name}")

    print(f"\nItem-Based Recommendations for User {user_test_ids[recommendations.index(i)]}:")
    for anime_id in item_recommendations:
        anime_name = anime.loc[anime['anime_id'] == anime_id, 'name'].values[0]
        print(f"- {anime_name}")

User-Based Recommendations for User 9:
- Soul Eater
- Inu x Boku SS
- Barakamon
- Aldnoah.Zero 2nd Season
- Death Billiards

Item-Based Recommendations for User 9:
- Ninja Mono
- Koisuru Tenshi Angelique: Kagayaki no Ashita
- Vexille: 2077 Nihon Sakoku
- Futari Daka
- Seisen Gakuin
User-Based Recommendations for User 22:
- Soul Eater
- Inu x Boku SS
- Barakamon
- Aldnoah.Zero 2nd Season
- Death Billiards

Item-Based Recommendations for User 22:
- Ninja Mono
- Koisuru Tenshi Angelique: Kagayaki no Ashita
- Vexille: 2077 Nihon Sakoku
- Futari Daka
- Seisen Gakuin
User-Based Recommendations for User 69:
- Soul Eater
- Inu x Boku SS
- Barakamon
- Aldnoah.Zero 2nd Season
- Death Billiards

Item-Based Recommendations for User 69:
- Ninja Mono
- Koisuru Tenshi Angelique: Kagayaki no Ashita
- Vexille: 2077 Nihon Sakoku
- Futari Daka
- Seisen Gakuin
